# Cell classification model interpretability with LIME 

**Keywords:  microscopy images, cell classification, EfficientNetV2, LIME, XAI**

The three categories of neuronal cells in this dataset, **shsy5y**, **astro** and **cort**,  are easy to classify. With or perhaps even without the aid of image processing  enhancements like CLAHE, it is possible to classify  the 520x704 resolution images [visually](#visualization). It therefore stands to reason that a good  deep learning model would also be able to easily classify the images.  
It would be interesting to see what features a trained deep learning model uses to identify the classes. To this end we can apply LIME [[1]](#ref1), a post hoc explainability technique that makes individual predictions of a black box model like a deep neural net, locally interpretable. More on LIME in [Part III](#PartIII).

In this exercise, we will train a deep learning EfficientNetV2 model to classify the neuronal cell images. We will then apply LIME on each class of image to get an insight into the model. 

In [ ]:
import numpy as np
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt
import cv2   # for CLAHE

import os
import random 
import glob
from itertools import product  # for confusion matrix plot
from itertools import islice
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# for LIME
from lime import lime_image
explainer = lime_image.LimeImageExplainer(random_state=2021)
from skimage.segmentation import mark_boundaries

import warnings
warnings.simplefilter("ignore")

print("TF version: ", tf.__version__)
print('Hub version:', hub.__version__)

#### Path for EfficientNet V2 model weights

In [ ]:
## Mapping copied from a "TF Hub for TF2: Retraining an image classifier" colab example
## For efficientnet class of models

def get_hub_url_and_isize(model_name, ckpt_type, hub_type):
  if ckpt_type == '1k':
    ckpt_type = ''  # json doesn't support empty string
  else:
    ckpt_type = '-' + ckpt_type  # add '-' as prefix
  
  hub_url_map = {
    'efficientnetv2-b0': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0/{hub_type}',
    'efficientnetv2-b1': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1/{hub_type}',
    'efficientnetv2-b2': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2/{hub_type}',
    'efficientnetv2-b3': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3/{hub_type}',
    'efficientnetv2-s':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s/{hub_type}',
    'efficientnetv2-m':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m/{hub_type}',
    'efficientnetv2-l':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l/{hub_type}',

    'efficientnetv2-b0-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0-21k/{hub_type}',
    'efficientnetv2-b1-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1-21k/{hub_type}',
    'efficientnetv2-b2-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2-21k/{hub_type}',
    'efficientnetv2-b3-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3-21k/{hub_type}',
    'efficientnetv2-s-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s-21k/{hub_type}',
    'efficientnetv2-m-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m-21k/{hub_type}',
    'efficientnetv2-l-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l-21k/{hub_type}',
    'efficientnetv2-xl-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-xl-21k/{hub_type}',

    'efficientnetv2-b0-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0-21k-ft1k/{hub_type}',
    'efficientnetv2-b1-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1-21k-ft1k/{hub_type}',
    'efficientnetv2-b2-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2-21k-ft1k/{hub_type}',
    'efficientnetv2-b3-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3-21k-ft1k/{hub_type}',
    'efficientnetv2-s-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s-21k-ft1k/{hub_type}',
    'efficientnetv2-m-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m-21k-ft1k/{hub_type}',
    'efficientnetv2-l-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l-21k-ft1k/{hub_type}',
    'efficientnetv2-xl-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-xl-21k-ft1k/{hub_type}',
      
    # efficientnetv1
    'efficientnet_b0': f'https://tfhub.dev/tensorflow/efficientnet/b0/{hub_type}/1',
    'efficientnet_b1': f'https://tfhub.dev/tensorflow/efficientnet/b1/{hub_type}/1',
    'efficientnet_b2': f'https://tfhub.dev/tensorflow/efficientnet/b2/{hub_type}/1',
    'efficientnet_b3': f'https://tfhub.dev/tensorflow/efficientnet/b3/{hub_type}/1',
    'efficientnet_b4': f'https://tfhub.dev/tensorflow/efficientnet/b4/{hub_type}/1',
    'efficientnet_b5': f'https://tfhub.dev/tensorflow/efficientnet/b5/{hub_type}/1',
    'efficientnet_b6': f'https://tfhub.dev/tensorflow/efficientnet/b6/{hub_type}/1',
    'efficientnet_b7': f'https://tfhub.dev/tensorflow/efficientnet/b7/{hub_type}/1',
  }
  
  image_size_map = {
    'efficientnetv2-b0': 224,
    'efficientnetv2-b1': 240,
    'efficientnetv2-b2': 260,
    'efficientnetv2-b3': 300,
    'efficientnetv2-s':  384,
    'efficientnetv2-m':  480,
    'efficientnetv2-l':  480,
    'efficientnetv2-xl':  512,
  
    'efficientnet_b0': 224,
    'efficientnet_b1': 240,
    'efficientnet_b2': 260,
    'efficientnet_b3': 300,
    'efficientnet_b4': 380,
    'efficientnet_b5': 456,
    'efficientnet_b6': 528,
    'efficientnet_b7': 600,
  }
  

  hub_url = hub_url_map.get(model_name + ckpt_type)
  image_size = image_size_map.get(model_name)
  return hub_url, image_size

In [ ]:
CONFIG = dict (
    seed = 2021,    
    num_classes = 3,
    train_val_split = 0.2,
    batch_size = 8,
    epochs = 3, 
    learning_rate = 1e-5,
    loss = 'sparse_categorical_crossentropy',
    metrics = 'sparse_categorical_accuracy',
    optimizer = tf.keras.optimizers.Adam(lr = 1e-4),
)


# Config for pretrained model 
CONFIG['model_type'] = 'efficientnetv2-b0'
CONFIG['ckpt_type'] = '1k'   # '1k', '21k-ft1k', '21k'
CONFIG['hub_type'] = 'feature-vector' 
hub_url, image_size = get_hub_url_and_isize(CONFIG['model_type'], CONFIG['ckpt_type'], CONFIG['hub_type'])
print(f'Hub URL: {hub_url}')
CONFIG['img_width'] = image_size
CONFIG['img_height'] = image_size
CONFIG['img_size'] = image_size
CONFIG['do_fine_tuning'] = True



def seed_everything(SEED):
    np.random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    #os.environ['TF_CUDNN_DETERMINISTIC'] = str(SEED)
    
seed_everything(CONFIG['seed'])

CSV_FILE = '../input/sartorius-cell-instance-segmentation/train.csv'
TRAIN_PATH = '../input/sartorius-cell-instance-segmentation/train/'

AUTOTUNE = tf.data.experimental.AUTOTUNE

TRAIN_MODEL = False # if False, use saved model for LIME; if true, train a new model

## Part I: EDA, data preprocessing (tf.data)

In [ ]:
col_list = ['id', 'cell_type']
df = pd.read_csv(CSV_FILE, usecols=col_list)
df['path'] = TRAIN_PATH + df['id'] + '.png'

df.rename(columns = {"cell_type": "label"}, inplace = True)
print(f'The dataset has {df.shape[0]} entries.\n')

CLASSES = df['label'].unique()
print(f'The labels are: {CLASSES[0]}, {CLASSES[1]}, {CLASSES[2]}')

In [ ]:
# dict to encode labels
label_dict = {'shsy5y': 0, 'astro': 1, 'cort':2}
df['label'] = df['label'].replace(label_dict)

# reverse dict to get back original labels
rev_label_dict = dict((v,k) for k,v in label_dict.items())

df.head()

#### Visualize each image class

In [ ]:
plt.figure(figsize=(16, 12))
for i in range(3):
    # get first id instance for each label
    # the image id's are: '0030fd0e6378', '0140b3c8f445', '01ae5a43a2ab'
    df_temp_ = df.loc[df.label == i].head(1)
    filename = df_temp_.iloc[0,2] # get path+filename
    
    # CLAHE to enhance image
    img = cv2.imread(filename)[..., 0]
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
    img2 = clahe.apply(img)
    
    ax = plt.subplot(1, 3, i + 1)
    plt.imshow(img2, cmap='hsv')
    plt.title(rev_label_dict.get(i))
    plt.axis('off')
    plt.tight_layout();

<a id='visualization'></a>

Based on the spacing and pattern, it is possible to visually identify an *astro* image from a *shsy5y* image from a *cort* image.
This is a descriptive summary of the three images:   
- *shsy5y* cells: numerous, dense and of uniform size
- *astro* cells: sparse with elongated processes that radiate from the center
- *cort* cells: clusters of larger cells with smaller cells/artifacts in the background

In [ ]:
X_train, X_valid = train_test_split(df, 
                                    test_size=CONFIG['train_val_split'], 
                                    random_state=CONFIG['seed'], 
                                    shuffle=True)


train_ds = tf.data.Dataset.from_tensor_slices((X_train.path.values, X_train.label.values))
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid.path.values, X_valid.label.values))

# print path/label
for path, label in train_ds.take(3):
    print ('Path: {}, Label: {}'.format(path, label))

In [ ]:
def preprocess_data_train(image_path, label):
    # load the raw data as a string
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0 
    # augment data in CPU
    img = tf.image.random_brightness(img, 0.3)
    img = tf.image.random_flip_left_right(img, seed=None)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.resize(img, [CONFIG['img_size'],CONFIG['img_size']])
    return img, label

def preprocess_data_valid(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.image.resize(img, [CONFIG['img_size'],CONFIG['img_size']])
    return img, label


# set num_parallel_calls to process multiple images in parallel
train_ds = train_ds.map(preprocess_data_train, num_parallel_calls=AUTOTUNE)
valid_ds = valid_ds.map(preprocess_data_valid, num_parallel_calls=AUTOTUNE)

In [ ]:
def configure_for_performance(ds, batch_size = 8):    
    ds = ds.shuffle(buffer_size=1024)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds


train_ds_batch = configure_for_performance(train_ds, CONFIG['batch_size'])
valid_ds_batch = valid_ds.batch(CONFIG['batch_size'])

#### Plot augmented images
A sanity check on the image pipeline where the images have been loaded, preprocessed (resized), batched and augmented (brightness, vertical/horizontal flips).

In [ ]:
image_batch, label_batch = next(iter(train_ds_batch))

plt.figure(figsize=(16, 12))
for i in range(6):
    ax = plt.subplot(2, 3, i + 1)
    plt.imshow(image_batch[i].numpy(), cmap='bone_r')
    label = label_batch[i].numpy()
    plt.title(rev_label_dict.get(label))
    plt.axis("off");

## Part II: Model training

In [ ]:
base_model = hub.KerasLayer(hub_url, trainable=True,
                           input_shape=(CONFIG['img_width'], CONFIG['img_height'], 3))

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.Dense(CONFIG['num_classes'], activation='softmax')
])


model.compile(
    optimizer=CONFIG['optimizer'],
    loss=CONFIG['loss'],
    metrics=CONFIG['metrics'])
model.summary()

In [ ]:
weight_path_save = 'best_model.hdf5'
last_weight_path = 'last_model.hdf5'

checkpoint = ModelCheckpoint(weight_path_save, 
                             monitor= 'val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode= 'min', 
                             save_weights_only = False)
checkpoint_last = ModelCheckpoint(last_weight_path, 
                             monitor= 'val_loss', 
                             verbose=1, 
                             save_best_only=False, 
                             mode= 'min', 
                             save_weights_only = False)


early = EarlyStopping(monitor= 'val_loss', 
                      mode= 'min', 
                      patience=5)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8, 
                                   patience=2, verbose=1, mode='auto', 
                                   epsilon=0.0001, cooldown=5, min_lr=0.00001)
callbacks_list = [checkpoint, checkpoint_last, early, reduce_lr]

In [ ]:
# plot train and validation curves

def plot_hist(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    acc = history.history['sparse_categorical_accuracy']
    val_acc = history.history['val_sparse_categorical_accuracy']
    epochs = range(1,len(loss)+1)

    fig = plt.figure(figsize=(9, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, linestyle='--', linewidth=3, color='red', alpha=0.7, label='Train Loss')
    plt.plot(epochs, val_loss, linestyle='dashdot', linewidth=2, color='green', alpha=0.8, label='Valid Loss')
    plt.xlim(1, max(plt.xlim()))
    plt.ylim(0,max(max(plt.ylim()), 0.01))
    plt.xlabel('Epochs', fontsize=11)
    plt.ylabel('Loss', fontsize=12)
    plt.title('Training/Validation Loss')
    plt.legend(fontsize=12)


    plt.subplot(1, 2, 2) 
    plt.plot(epochs, acc, linestyle='--', linewidth=3, color='red', alpha=0.7, label='Train Acc')
    plt.plot(epochs, val_acc, linestyle='solid', linewidth=4, color='green', alpha=0.8, label='Valid Acc') 
    plt.xlim(1, max(plt.xlim()))
    plt.ylim(min(min(plt.ylim()), 0.98),1)
    plt.xlabel('Epochs', fontsize=11)
    plt.ylabel('Accuracy', fontsize=12)
    plt.title('Training/Validation Accuracy')
    plt.legend(fontsize=12)
    
    plt.tight_layout()
    plt.show()

In [ ]:
if TRAIN_MODEL:
    history = model.fit(train_ds_batch, 
                        validation_data = valid_ds_batch, 
                        epochs = CONFIG['epochs'], 
                        callbacks = callbacks_list,
                       )
    plot_hist(history)

## Part III: Explainability with LIME

### Evaluating Model on Validation Set

In [ ]:
def plot_confusion_matrix(cm, 
                          classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Purples):
   

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy  
    
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=15)
    plt.yticks(tick_marks, classes, fontsize=15)
    
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'
           .format(accuracy, misclass),fontsize=15)
    plt.ylabel('True label', fontsize=15)
    plt.title(title, fontsize=22);
    plt.colorbar()

    thresh = cm.max() / 2.0
    for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
   

In [ ]:
if TRAIN_MODEL: 
    model.load_weights(weight_path_save) # load the best model 
else: 
    model.load_weights('../input/sartorius-best-model/best_model.hdf5') # load a previously trained model
    model.summary()

In [ ]:
pred_prob = model.predict(valid_ds_batch, workers=4, verbose = True)
pred_labels = np.argmax(pred_prob, axis=-1)

valid_labels = np.concatenate([y.numpy() for x, y in valid_ds_batch], axis=0)
print('\n', classification_report(valid_labels, pred_labels ))

In [ ]:
cm = confusion_matrix(valid_labels, pred_labels)
classes = CLASSES
title = 'Confusion matrix of results'
plot_confusion_matrix(cm, classes, title);

<a id='PartIII'></a>

### LIME (Local Interpretable Model-Agnostic Explanations)

There is a trade-off between model interpretability and accuracy: simple models are generally more interpretable while complex models have lower interpretability but higher accuracy. Deep learning models are black box models so in this trade-off they would also be categorized as having lower interpretability/higher accuracy.

To make deep learning models more explainable we can apply post hoc methods like LIME. The intuition behind LIME is to perturb the local area around a single data instance and then apply a simple, intrinsically interpretable, surrogate linear model to this neighborhood. This helps us interpret the outcome for an individual data point, but not the global model. LIME can be used for text, tabular or image data.
For image data specifically, the procedure can  be outlined  as follows:

- generate multiple images from a given image by perturbing the image
  (perturbing: randomly selecting superpixels from the given image)
- use the trained model to predict the class for each of the generated images
- weight each image perturbation as a function of its proximity to the original image
- fit a weighted linear model (*explainer*) with the perturbed images as input and prediction as response
- each superpixel in the image is ranked based on how much it contributes to the class

In this exercise, we will look at three images, one from each class. Since the classifer we trained was 100% accurate, all three images are true positive.

#### Select three images

In [ ]:
# select a batch of images 
image_iter_2, label_iter_2 = next(islice(valid_ds_batch, 2, None)) # access the 2nd batch from the iterator

image_iter_2 = tf.expand_dims(image_iter_2, 0)
label_iter_2 = label_iter_2.numpy()

# print labels for the batch of 8 images 
print('The labels for the selected batch of 8 images are: ', label_iter_2)

In [ ]:
# select an image for each unique label
image_idx_selected = [1,4,0]

print(f'The selected images are labeled: ')
for i in image_idx_selected:
    print(f'{rev_label_dict.get(label_iter_2[i])}')

In [ ]:
def explain_image_by_lime(image, label):
    explanation = explainer.explain_instance(image, model.predict, 
                                             top_labels=3, hide_color=0, num_samples=1000)
    
    # show the top 3 superpixels that contribute to the class; do not show the rest of the image
    temp_1, mask_1 = explanation.get_image_and_mask(explanation.top_labels[0], 
                                                positive_only=True, num_features=3, hide_rest=True)

    # show the top 3 positive or negative superpixels; make the entire image visible
    temp_2, mask_2 = explanation.get_image_and_mask(explanation.top_labels[0], 
                                                positive_only=False, num_features=3, hide_rest=False)
    
  

    fig = plt.figure(figsize=(16, 12), constrained_layout=True)    
    plt.subplot(1, 3, 1)
    plt.imshow(image, cmap='gray')
    plt.title('original image', fontsize=14)
    plt.axis('off')
    plt.subplot(1, 3, 2)
    plt.imshow(mark_boundaries(temp_1, mask_1))
    plt.title('positive superpixels', fontsize=14)
    plt.axis('off')
    plt.subplot(1, 3, 3)
    plt.imshow(mark_boundaries(temp_2, mask_2), cmap='gray')
    plt.title('positive & negative superpixels', fontsize=14)
    plt.axis('off')
    plt.suptitle(label, y=0.8, fontsize=18)
    plt.show()

#### Image 1

In [ ]:
image = image_iter_2[0][1].numpy().astype('double')
label = rev_label_dict.get(label_iter_2[1])
print(f'The image selected is labeled: {label}\n')

explain_image_by_lime(image, label)

On the left is the original *shsy5y* image. The image is slightly blurred near the edges.  

The image in the center *explains* the prediction. It shows the superpixels that most positively contribute to the correct classification of the image. These superpixels which are from the least blurred parts of the image, show the evenly distributed *shsy5y* cells.   

On the right the green superpixel is used to positively identify the class. The red negatively contributes to the class. We can see that the regions corresponding to the red in the original image are blurred edges.

#### Image 2

In [ ]:
image = image_iter_2[0][4].numpy().astype('double')
label = rev_label_dict.get(label_iter_2[4])
print(f'The image selected is labeled: {label}\n')

explain_image_by_lime(image, label)

On the left we see the original *astro* image. The lower left corner is blurred/smudged and appears empty.   

In the center, we see the superpixels that most positively contribute to the *astro* classification. One superpixel captures the large blank spaces that surround the elongated cell projections. This is the characterizing visual  feature that I would also use to classify an image as *astro*. Note the superpixel at the lower left corner; that is the section where the image is smudged and is probably being treated as a blank space.

On the right we see the superpixels which contribute positively (green) to the identification and negatively (red). The red superpixel includes a region dense with cells that do not appear to have the characteristic elongated radiations.

#### Image 3

In [ ]:
image = image_iter_2[0][0].numpy().astype('double')
label = rev_label_dict.get(label_iter_2[0])
print(f'The image selected is labeled: {label}\n')

explain_image_by_lime(image, label)

In the original image on the left, we see larger *cort* cell clusters with smaller cells/artifacts scattered in the background.

In the center image, the two superpixels that most strongly contribute to the positive identification highlight the background cells/artifacts we see in *cort* images but not *shsy5y* images.

On the right, the green superpixel positively identifies the image; the red negatively identifies the class. It is possible that the red superpixel, which juxtaposes a cell cluster, is being identified as the negative space in an *astro* cell.

## Summary

The deep learning model appears to have used some of the same visual cues I used to classify the images. To wit: 
- the *shsy5y* cells were identified by the size and distribution of the cells.  
- the *astro* cells were identified by the sparse negative space between cells.  
- the *cort* cells are also identified by the background.  

The reader is invited to explore and interpret LIME explanations with a different set of images.

## References

<a name='ref1'></a>[1] [M. Ribeiro, S. Singh, C. Guestrin,
'"Why Should I Trust You?": Explaining the Predictions of Any Classifier.' arXiv:1602.04938](https://arxiv.org/abs/1602.04938)

**Author: Meena Mani**        
**Date: January 2022**